In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import psycopg2

In [2]:
from dask.distributed import Client
from dask_kubernetes.operator import KubeCluster
cluster = KubeCluster(name='cluster2', image='ghcr.io/patmagauran/mldocker-base:main',env={'EXTRA_CONDA_PACKAGES':'gcsfs'}, resources={"requests": {"memory": "3Gi"}, "limits": {"memory": "6Gi"}}  )
#cluster = KubeCluster.from_name(name="foo")
#cluster.adapt(minimum=4, maximum=8)
cluster.scale(5)
# Scale up: connect to your own cluster with more resources
# see http://dask.pydata.org/en/latest/setup.html


W1127 04:47:28.384821   20879 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1127 04:47:39.666901   20909 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke


In [3]:
client = Client(cluster)
client

<Client: 'tcp://10.56.1.2:8786' processes=5 threads=10, memory=30.00 GiB>

In [44]:
df = dd.read_parquet("gcs://ds352-data/processed-1")
#df = df.categorize()
#df._meta.dtypes.to_dict()

In [45]:
#df = df.fillna(value = {'Bicycle': False})
#df.Bicycle.value_counts().compute()

In [46]:
#df.Bicycle.isna().any().compute()

In [47]:
bicycles = df[df.Bicycle]
bicycles

,Crash Record Number,County Code,County Name,Municipality Code,Municipality Name,District Code,District Name,Police Agency Code,Police Agency Name,Crash Year,Crash Month,Day of Week,Time of Day,Hour of Day,Illumination,Weather,Road Condition,Collision Type,Work Zone Type,Work Zone Location,Intersection Type,Traffic Control Device,Location Type,Urban / Rural,Fatality Count,Injury Count,Person Count,Total Unit Count,School Bus Indicator,School Zone Indicator,Latitude,Longitude,Arrival Time,Dispatch Time,Construction Zone Speed Limit,Estimated Hours Closed,Lane Closed,Lane Closure Direction,Notify Highway Maintenance,Special Jurisdiction,Traffic Control Device Function,Traffic Detour,Workers Present,Work Zone Close/Detour,Work Zone Flagger,Work Zone Law Officer,Work Zone Closure,Work Zone Moving,Work Zone Other,Work Zone Shoulder/Median,Vehicle Count,Automobile Count,Motorcycle Count,Bus Count,Small Truck Count,Heavy Truck Count,SUV Count,Van Count,Bicycle Count,Suspected Serious Injury Count,Suspected Minor Injury Count,Possible Injury Count,Total Injury Count,Unknown Injury Degree Count,Unknown Injury Person Count,16 Year Old Driver Count,17 Year Old Driver Count,18 Year Old Driver Count,19 Year Old Driver Count,20 Year Old Driver Count,50-64 Year Old Driver Count,65-74 Year Old Driver Count,75 Plus Year Old Driver Count,Unbelted Occupant Count,Unbelted Death Count,Unbelted Suspected Serious Injury Count,Belted Death Count,Belted Suspected Serious Injury Count,Motorcycle Death Count,Motorcycle Suspected Serious Injury Count,Bicycle Death Count,Bicycle Suspected Serious Injury Count,Pedestrian Count,Pedestrian Death Count,Pedestrian Suspected Serious Injury Count,Maximum Severity Level,Commercial Vehicle Count,Latitude (Decimal),Longitude (Decimal),Roadway Surface Type,Interstate,State Road,Local Road Only,Turnpike,Wet Road,Snow Slush Road,Icy Road,Sudden Deer,Shoulder Related,Rear End,Head On / Opposite Direction Side Swipe,Hit Fixed Object,Single Vehicle Run Off Road,Work Zone,Property Damage Only,Injury or Fatal,Fatal or Suspected Serious Injury,Injury,Fatal,Non Intersection,Intersection,Signalized Intersection,Stop Controlled Intersection,Unsignalized Intersection,School Bus,School Zone,Hit Deer,Hit Tree / Shrub,Hit Embankment,Hit Pole,Hit Guide Rail,Hit Guide Rail End,Hit Barrier,Hit Bridge,Overturned,Motorcycle,Bicycle,Heavy Truck Related,Vehicle Failure,Train / Trolley,Phantom Vehicle,Alcohol Related,Drinking Driver,Underage Drinking Driver,Unlicensed,Distracted,Cell Phone,No Clearance,Running Red Light,Tailgating,Cross Median,Curved Road,Curve Driver Error,Limit 65 MPH,Speeding,Speeding Related,Aggressive Driving,Fatigue / Asleep,Driver 16 Years Old,Driver 17 Years Old,Driver 18 Years Old,Driver 19 Years Old,Driver 20 Years Old,Driver 50 - 64 Years Old,Driver 65 - 74 Years Old,Driver 75 Plus,Unbelted,Pedestrian,Commercial Vehicle,PSP Reported,NHTSA Aggressive Driving,Deer Related,Illumination Dark,Running Stop Sign,Train,Trolley,Hit Parked Vehicle,Fire In Vehicle,Vehicle Towed,Hazardous Truck,Suspected Serious Injury,Suspected Minor Injury,Possible Injury,Motorcycle Drinking Driver,Drug Related,Illegal Drug Related,School Bus Unit,Drugged Driver,Impaired Driver,Geographic Location of the Crash
npartitions=28,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,Int32,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],Int8,Int8,int32,Int8,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],UInt8,UInt8,UInt8,UInt8,object,object,object,object,Int16,Int16,Int8,Int8,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unk

In [48]:
nonBicycles = df[~df.Bicycle]
nonBicycles

,Crash Record Number,County Code,County Name,Municipality Code,Municipality Name,District Code,District Name,Police Agency Code,Police Agency Name,Crash Year,Crash Month,Day of Week,Time of Day,Hour of Day,Illumination,Weather,Road Condition,Collision Type,Work Zone Type,Work Zone Location,Intersection Type,Traffic Control Device,Location Type,Urban / Rural,Fatality Count,Injury Count,Person Count,Total Unit Count,School Bus Indicator,School Zone Indicator,Latitude,Longitude,Arrival Time,Dispatch Time,Construction Zone Speed Limit,Estimated Hours Closed,Lane Closed,Lane Closure Direction,Notify Highway Maintenance,Special Jurisdiction,Traffic Control Device Function,Traffic Detour,Workers Present,Work Zone Close/Detour,Work Zone Flagger,Work Zone Law Officer,Work Zone Closure,Work Zone Moving,Work Zone Other,Work Zone Shoulder/Median,Vehicle Count,Automobile Count,Motorcycle Count,Bus Count,Small Truck Count,Heavy Truck Count,SUV Count,Van Count,Bicycle Count,Suspected Serious Injury Count,Suspected Minor Injury Count,Possible Injury Count,Total Injury Count,Unknown Injury Degree Count,Unknown Injury Person Count,16 Year Old Driver Count,17 Year Old Driver Count,18 Year Old Driver Count,19 Year Old Driver Count,20 Year Old Driver Count,50-64 Year Old Driver Count,65-74 Year Old Driver Count,75 Plus Year Old Driver Count,Unbelted Occupant Count,Unbelted Death Count,Unbelted Suspected Serious Injury Count,Belted Death Count,Belted Suspected Serious Injury Count,Motorcycle Death Count,Motorcycle Suspected Serious Injury Count,Bicycle Death Count,Bicycle Suspected Serious Injury Count,Pedestrian Count,Pedestrian Death Count,Pedestrian Suspected Serious Injury Count,Maximum Severity Level,Commercial Vehicle Count,Latitude (Decimal),Longitude (Decimal),Roadway Surface Type,Interstate,State Road,Local Road Only,Turnpike,Wet Road,Snow Slush Road,Icy Road,Sudden Deer,Shoulder Related,Rear End,Head On / Opposite Direction Side Swipe,Hit Fixed Object,Single Vehicle Run Off Road,Work Zone,Property Damage Only,Injury or Fatal,Fatal or Suspected Serious Injury,Injury,Fatal,Non Intersection,Intersection,Signalized Intersection,Stop Controlled Intersection,Unsignalized Intersection,School Bus,School Zone,Hit Deer,Hit Tree / Shrub,Hit Embankment,Hit Pole,Hit Guide Rail,Hit Guide Rail End,Hit Barrier,Hit Bridge,Overturned,Motorcycle,Bicycle,Heavy Truck Related,Vehicle Failure,Train / Trolley,Phantom Vehicle,Alcohol Related,Drinking Driver,Underage Drinking Driver,Unlicensed,Distracted,Cell Phone,No Clearance,Running Red Light,Tailgating,Cross Median,Curved Road,Curve Driver Error,Limit 65 MPH,Speeding,Speeding Related,Aggressive Driving,Fatigue / Asleep,Driver 16 Years Old,Driver 17 Years Old,Driver 18 Years Old,Driver 19 Years Old,Driver 20 Years Old,Driver 50 - 64 Years Old,Driver 65 - 74 Years Old,Driver 75 Plus,Unbelted,Pedestrian,Commercial Vehicle,PSP Reported,NHTSA Aggressive Driving,Deer Related,Illumination Dark,Running Stop Sign,Train,Trolley,Hit Parked Vehicle,Fire In Vehicle,Vehicle Towed,Hazardous Truck,Suspected Serious Injury,Suspected Minor Injury,Possible Injury,Motorcycle Drinking Driver,Drug Related,Illegal Drug Related,School Bus Unit,Drugged Driver,Impaired Driver,Geographic Location of the Crash
npartitions=28,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,Int32,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],Int8,Int8,int32,Int8,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],UInt8,UInt8,UInt8,UInt8,object,object,object,object,Int16,Int16,Int8,Int8,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown],category[unk

In [49]:
SampleFrac = (bicycles.shape[0]/nonBicycles.shape[0]) * 1.0

In [50]:
nonBicyclesSampled = nonBicycles.sample(frac=SampleFrac.compute(), random_state = 4)
#nonBicyclesSampled.shape[0].compute()

In [51]:
final = dd.concat([bicycles,nonBicyclesSampled])
#final.shape[0].compute()

In [52]:
#final._meta.dtypes.to_dict()

In [53]:
#final['School Bus Indicator'].value_counts().compute()

In [54]:
final = final.astype({'School Bus Indicator': 'bool',
 'School Zone Indicator': 'bool',
  'Interstate': 'bool',
 'State Road': 'bool',
 'Local Road Only': 'bool',
 'Turnpike': 'bool',
 'Wet Road': 'bool',
 'Snow Slush Road': 'bool',
 'Icy Road': 'bool',
 'Sudden Deer': 'bool',
 'Shoulder Related': 'bool',
 'Rear End': 'bool',
 'Head On / Opposite Direction Side Swipe': 'bool',
 'Hit Fixed Object': 'bool',
 'Single Vehicle Run Off Road': 'bool',
 'Work Zone': 'bool',
 'Property Damage Only': 'bool',
 'Injury or Fatal': 'bool',
 'Fatal or Suspected Serious Injury': 'bool',
 'Injury': 'bool',
 'Fatal': 'bool',
 'Non Intersection': 'bool',
 'Intersection': 'bool',
 'Signalized Intersection': 'bool',
 'Stop Controlled Intersection': 'bool',
 'Unsignalized Intersection': 'bool',
 'School Bus': 'bool',
 'School Zone': 'bool',
 'Hit Deer': 'bool',
 'Hit Tree / Shrub': 'bool',
 'Hit Embankment': 'bool',
 'Hit Pole': 'bool',
 'Hit Guide Rail': 'bool',
 'Hit Guide Rail End': 'bool',
 'Hit Barrier': 'bool',
 'Hit Bridge': 'bool',
 'Overturned': 'bool',
 'Motorcycle': 'bool',
 'Bicycle': 'bool',
 'Heavy Truck Related': 'bool',
 'Vehicle Failure': 'bool',
 'Train / Trolley': 'bool',
 'Phantom Vehicle': 'bool',
 'Alcohol Related': 'bool',
 'Drinking Driver': 'bool',
 'Underage Drinking Driver': 'bool',
 'Unlicensed': 'bool',
 'Distracted': 'bool',
 'Cell Phone': 'bool',
 'No Clearance': 'bool',
 'Running Red Light': 'bool',
 'Tailgating': 'bool',
 'Cross Median': 'bool',
 'Curved Road': 'bool',
 'Curve Driver Error': 'bool',
 'Limit 65 MPH': 'bool',
 'Speeding': 'bool',
 'Speeding Related': 'bool',
 'Aggressive Driving': 'bool',
 'Fatigue / Asleep': 'bool',
 'Driver 16 Years Old': 'bool',
 'Driver 17 Years Old': 'bool',
 'Driver 18 Years Old': 'bool',
 'Driver 19 Years Old': 'bool',
 'Driver 20 Years Old': 'bool',
 'Driver 50 - 64 Years Old': 'bool',
 'Driver 65 - 74 Years Old': 'bool',
 'Driver 75 Plus': 'bool',
 'Unbelted': 'bool',
 'Pedestrian': 'bool',
 'Commercial Vehicle': 'bool',
 'PSP Reported': 'bool',
 'NHTSA Aggressive Driving': 'bool',
 'Deer Related': 'bool',
 'Illumination Dark': 'bool',
 'Running Stop Sign': 'bool',
 'Train': 'bool',
 'Trolley': 'bool',
 'Hit Parked Vehicle': 'bool',
 'Fire In Vehicle': 'bool',
 'Vehicle Towed': 'bool',
 'Hazardous Truck': 'bool',
 'Suspected Serious Injury': 'bool',
 'Suspected Minor Injury': 'bool',
 'Possible Injury': 'bool',
 'Motorcycle Drinking Driver': 'bool',
 'Drug Related': 'bool',
 'Illegal Drug Related': 'bool',
 'School Bus Unit': 'bool',
 'Drugged Driver': 'bool',
 'Impaired Driver': 'bool'})

In [55]:
#final = final.compute()
#final = dd.from_pandas(final, npartitions=5)

In [56]:
import pyarrow as pa
final.to_parquet("gs://ds352-data/balanced-1",
             schema={
              "Municipality Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Municipality Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "District Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "District Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Police Agency Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Police Agency Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False)                 
             })

In [57]:
#final.head()